<a href="https://colab.research.google.com/github/byuccl/digital_design_colab/blob/dev/Lab5_State_Machines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Setup**
Click the Play Button. No need to expand. Do not touch this. 
<details><summary>Packages to be downloaded</summary>
Things it needs to install:  <br>

 *    Verilator- the simulator  <br>
 *    PyGithub - Helps with simulation    <br>
 *    VCD - Generates waveforms from `.vcd` files  <br>
 *    Widgets - Each widget must first be generated then will be loaded when played
</details>

In [ ]:
#@title Setup
!cd /content
!git clone https://github.com/yne/vcd.git
!make -C /content/vcd
%cd /content/vcd
!make install
%cd /
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
import requests

!mkdir -p /content/tmp_code
#Creates a text document using the Raw github url. This text document will be used as a .py file for imports
def import_text(text):
  url = "https://raw.githubusercontent.com/byuccl/digital_design_colab/master/Labs/state_machine_lab/files/%s" % text
  resp = requests.get(url)
  with open(text, 'wb') as f:
    f.write(resp.content)

import_text("import_all.py")
from import_all import *
import_source()
import_packages()
from simulation import *
from vcd2wd import *
!apt-get install verilator >/dev/null
def quiz_func(a):
  quiz_bot.quiz_func(a)
!pip install --upgrade git+https://github.com/anon36424/nb_js_diagrammers.git
%load_ext nb_js_diagrammers

# One Shot 

One large problem when dealing with inputs is that they are read every single cycle.  
It is impossible to press a button for a single clock cycle. So if we press the button
it may be pressed for thousands of clock cycles. This is where a one shot helps us.  

It only reads the button press for cycle. It does this by converting the button into a different signal which after one cycle goes low. Releasing and pressing again can reset the oneshot.

One Shot should have 3 states, IDLE, ONESHOT, WAIT.  
    IDLE: Waiting for an input. When in goes high, transitions to ONESHOT.
    ONESHOT: Input received, out is high for one cycle. Goes to WAIT.
    WAIT: Waits for the input to go low. Out is low. When out goes to low, transitions to IDLE.  

| Module Name: | OneShot |||
| -- --------- | ----------- |--|--|
| Port Name      | Direction       |Width|Function|
| reset 	|Input 	|1| Active high reset|
| in 	|Input 	|1| Signal to One Shot|
| out |Output |1| One Shot signal|
| clk 	|Input 	|1| Clock Signal |


# Safe Combination State Machine

For this lab you will design a combination lock state machine.
States:  



<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab/main/State_Machine/media/StateDiagram.svg"
width="500" height="300" style="display: block; margin: 0 auto " />
<br>

Intermediate Signals:  
CODE: If the correct code for the current state is input, then CODE is high, else, CODE is low.  
Q: If any button is pressed, Q is high. This represents entering the code.  


State Machine:  

IDLE: The state machine will wait for BTNC to be pressed.

COMBINATION_ONE: IF Q is low, nothing happens. If Q is High and Code is high, the correct code has been entered and go to next state. If Q is High and Code is Low, the incorrect code has been entered so go back to the IDLE state. In this state, LED should be given a value of `1`. 

COMBINATION_TWO: IF Q is low, nothing happens. If Q is High and Code is high, the correct code has been entered and go to next state. If Q is High and Code is Low, the incorrect code has been entered so go back to the IDLE state.  LED should be given a value of `2`. 


COMBINATION_THREE: IF Q is low, nothing happens. If Q is High and Code is high, the correct code has been entered and go to next state. If Q is High and Code is Low, the incorrect code has been entered so go back to the IDLE state.  LED should be given a value of `3`. 

OPEN: After the 3 codes have been entered, the safe will 'open' and show an LED code. If BTNC is pressed the safe will go back to IDLE state.  


| Module Name: | State Machine |||
| -- --------- | ----------- |--|--|
| Port Name      | Direction       |Width|Function|
|clk 	|Input 	|1| Clock used for timing|
|btnu 	|Input 	|1| Button Up|
|btnd 	|Input 	|1| Button Down|
|btnc 	|Input 	|1| Button Center|
|btnl 	|Input 	|1| Button Left|
|btnr 	|Input 	|1| Button Right|
|sw 	|Input 	|16 | Input used for the code|
| led | Output | 16 | Output for the State Machine |



 
Specifications:

Q: If btnu, btnd, btnc, btnr, or btnl are high, then Q is high.

CODE:  
Combination 1: btnd, and sw = `16'b0000000000000001`

Combination 2: btnr and sw = `16'b1111001111001111`

Combination 3: btnc and sw = `16'b0100101010100111`


## Creating a Module

In [ ]:
#@title
createSimulationWorkSpace("tmp_code/statemachine")

In [ ]:
#@title Create WaveDrom
df2wd("sm")

In [ ]:
#@title Show WaveDrom
%%wavedrom_magic -h 200 -o /content/tmp_code/sm.txt
---

###XDC Files

In [ ]:
#Set this up yourself
%%bash -c 'cat > /content/tmp_code/xdc.xdc'
###################################################################
# basys3_220.xdc
#
# This is a master constraints file for laboratory assignments used
# at BYU for ECEN 220.
#
# You should uncomment those lines that define ports that you used
# in your top-level design. You should also change the name of the
# port in the .xdc file to match your corresponding top-level port.
#
###################################################################

# ## Clock
# set_property -dict { PACKAGE_PIN W5   IOSTANDARD LVCMOS33 } [get_ports clk]
# create_clock -add -name sys_clk_pin -period 10.00 -waveform {0 5} [get_ports clk]

# ## Buttons
# set_property -dict { PACKAGE_PIN U18   IOSTANDARD LVCMOS33 } [get_ports { btnc }];
# set_property -dict { PACKAGE_PIN T18   IOSTANDARD LVCMOS33 } [get_ports { btnu }];
# set_property -dict { PACKAGE_PIN W19   IOSTANDARD LVCMOS33 } [get_ports { btnl }];
# set_property -dict { PACKAGE_PIN T17   IOSTANDARD LVCMOS33 } [get_ports { btnr }];
# set_property -dict { PACKAGE_PIN U17   IOSTANDARD LVCMOS33 } [get_ports { btnd }];

# ## Switches
# set_property -dict { PACKAGE_PIN V17   IOSTANDARD LVCMOS33 } [get_ports { sw[0] }];
# set_property -dict { PACKAGE_PIN V16   IOSTANDARD LVCMOS33 } [get_ports { sw[1] }];
# set_property -dict { PACKAGE_PIN W16   IOSTANDARD LVCMOS33 } [get_ports { sw[2] }];
# set_property -dict { PACKAGE_PIN W17   IOSTANDARD LVCMOS33 } [get_ports { sw[3] }];
# set_property -dict { PACKAGE_PIN W15   IOSTANDARD LVCMOS33 } [get_ports { sw[4] }];
# set_property -dict { PACKAGE_PIN V15   IOSTANDARD LVCMOS33 } [get_ports { sw[5] }];
# set_property -dict { PACKAGE_PIN W14   IOSTANDARD LVCMOS33 } [get_ports { sw[6] }];
# set_property -dict { PACKAGE_PIN W13   IOSTANDARD LVCMOS33 } [get_ports { sw[7] }];
# set_property -dict { PACKAGE_PIN V2    IOSTANDARD LVCMOS33 } [get_ports { sw[8] }];
# set_property -dict { PACKAGE_PIN T3    IOSTANDARD LVCMOS33 } [get_ports { sw[9] }];
# set_property -dict { PACKAGE_PIN T2    IOSTANDARD LVCMOS33 } [get_ports { sw[10] }];
# set_property -dict { PACKAGE_PIN R3    IOSTANDARD LVCMOS33 } [get_ports { sw[11] }];
# set_property -dict { PACKAGE_PIN W2    IOSTANDARD LVCMOS33 } [get_ports { sw[12] }];
# set_property -dict { PACKAGE_PIN U1    IOSTANDARD LVCMOS33 } [get_ports { sw[13] }];
# set_property -dict { PACKAGE_PIN T1    IOSTANDARD LVCMOS33 } [get_ports { sw[14] }];
# set_property -dict { PACKAGE_PIN R2    IOSTANDARD LVCMOS33 } [get_ports { sw[15] }];

# # LEDs
# set_property -dict { PACKAGE_PIN U16   IOSTANDARD LVCMOS33 } [get_ports { led[0] }];
# set_property -dict { PACKAGE_PIN E19   IOSTANDARD LVCMOS33 } [get_ports { led[1] }];
# set_property -dict { PACKAGE_PIN U19   IOSTANDARD LVCMOS33 } [get_ports { led[2] }];
# set_property -dict { PACKAGE_PIN V19   IOSTANDARD LVCMOS33 } [get_ports { led[3] }];
# set_property -dict { PACKAGE_PIN W18   IOSTANDARD LVCMOS33 } [get_ports { led[4] }];
# set_property -dict { PACKAGE_PIN U15   IOSTANDARD LVCMOS33 } [get_ports { led[5] }];
# set_property -dict { PACKAGE_PIN U14   IOSTANDARD LVCMOS33 } [get_ports { led[6] }];
# set_property -dict { PACKAGE_PIN V14   IOSTANDARD LVCMOS33 } [get_ports { led[7] }];
# set_property -dict { PACKAGE_PIN V13   IOSTANDARD LVCMOS33 } [get_ports { led[8] }];
# set_property -dict { PACKAGE_PIN V3    IOSTANDARD LVCMOS33 } [get_ports { led[9] }];
# set_property -dict { PACKAGE_PIN W3    IOSTANDARD LVCMOS33 } [get_ports { led[10] }];
# set_property -dict { PACKAGE_PIN U3    IOSTANDARD LVCMOS33 } [get_ports { led[11] }];
# set_property -dict { PACKAGE_PIN P3    IOSTANDARD LVCMOS33 } [get_ports { led[12] }];
# set_property -dict { PACKAGE_PIN N3    IOSTANDARD LVCMOS33 } [get_ports { led[13] }];
# set_property -dict { PACKAGE_PIN P1    IOSTANDARD LVCMOS33 } [get_ports { led[14] }];
# set_property -dict { PACKAGE_PIN L1    IOSTANDARD LVCMOS33 } [get_ports { led[15] }];

## Segment Display
#set_property -dict { PACKAGE_PIN W7   IOSTANDARD LVCMOS33 } [get_ports {segment[0]}]
#set_property -dict { PACKAGE_PIN W6   IOSTANDARD LVCMOS33 } [get_ports {segment[1]}]
#set_property -dict { PACKAGE_PIN U8   IOSTANDARD LVCMOS33 } [get_ports {segment[2]}]
#set_property -dict { PACKAGE_PIN V8   IOSTANDARD LVCMOS33 } [get_ports {segment[3]}]
#set_property -dict { PACKAGE_PIN U5   IOSTANDARD LVCMOS33 } [get_ports {segment[4]}]
#set_property -dict { PACKAGE_PIN V5   IOSTANDARD LVCMOS33 } [get_ports {segment[5]}]
#set_property -dict { PACKAGE_PIN U7   IOSTANDARD LVCMOS33 } [get_ports {segment[6]}]

##Anodes 
#set_property -dict { PACKAGE_PIN U2   IOSTANDARD LVCMOS33 } [get_ports {anode[0]}]
#set_property -dict { PACKAGE_PIN U4   IOSTANDARD LVCMOS33 } [get_ports {anode[1]}]
#set_property -dict { PACKAGE_PIN V4   IOSTANDARD LVCMOS33 } [get_ports {anode[2]}]
#set_property -dict { PACKAGE_PIN W4   IOSTANDARD LVCMOS33 } [get_ports {anode[3]}]
##



## Compiling with the F4PGA Toolchain


### Installing the Toolchain

In [ ]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()

In [ ]:
!apt install -y git wget xz-utils

In [ ]:
%%bash
cd /content
git clone https://github.com/chipsalliance/f4pga-examples
cd f4pga-examples
wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O conda_installer.sh


In [ ]:
#Creates the Conda Environment
%%bash
cd /content/f4pga-examples
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
bash conda_installer.sh -u -b -p $F4PGA_INSTALL_DIR/$FPGA_FAM/conda;
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
conda env create -f $FPGA_FAM/environment.yml

In [ ]:
#Download F4PGA Arch Defs
%%bash
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
conda activate xc7
mkdir -p $F4PGA_INSTALL_DIR/xc7/install

F4PGA_TIMESTAMP='20220907-210059'
F4PGA_HASH='66a976d'
export F4PGA_PACKAGES='install-xc7 xc7a50t_test'

for PKG in $F4PGA_PACKAGES; do
  wget -qO- https://storage.googleapis.com/symbiflow-arch-defs/artifacts/prod/foss-fpga-tools/symbiflow-arch-defs/continuous/install/${F4PGA_TIMESTAMP}/symbiflow-arch-defs-${PKG}-${F4PGA_HASH}.tar.xz | tar -xJC $F4PGA_INSTALL_DIR/${FPGA_FAM}
done

### Compiling with the Toolchain

In [ ]:
#This creates the Makefile
with open("/content/tmp_code/Makefile", "w") as f:
  f.write("""current_dir := ${CURDIR}
TARGET := basys3

TOP := StateMachine

XDC := ${current_dir}/*.xdc

SOURCES := $(wildcard ${current_dir}/*.v ${current_dir}/*.sv)

include /content/f4pga-examples/common/common.mk
""")

In [ ]:
#Make the project
%%bash
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
export TARGET=basys3
export FOLDER=State_Machine
export FILES="Makefile xdc.xdc StateMachine.sv OneShot.sv"
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
mkdir -p /content/"$FOLDER"
cd /content/tmp_code
for file in $FILES
do 
    cp $file /content/"$FOLDER"/$file
done
conda activate xc7
cd /content/$FOLDER
cp /content/tmp_code/errorFeedback.py ./
time SURELOG_CMD="-parse" make 2> ./error.txt 1>./compile.txt
python errorFeedback.py

## Testing it on the board

You can use this configuration file and the program openOCD to download the bitstream to your board. You will need to download this file and the bitstream to your local machine.

In [ ]:
#@title Create File for openocd
folder = "State_Machine"
bitfile = "StateMachine"

with open(f"/content/{folder}.cfg", "w") as f:
  f.write("""interface ftdi
ftdi_device_desc "Digilent USB Device"
ftdi_vid_pid 0x0403 0x6010
# channel 1 does not have any functionality
ftdi_channel 0
# just TCK TDI TDO TMS, no reset
ftdi_layout_init 0x0088 0x008b
reset_config none
adapter_khz 10000

source [find cpld/xilinx-xc7.cfg]
source [find cpld/jtagspi.cfg]
init

puts [irscan xc7.tap 0x09]

set xc7a35t "0362D093"
set xc7a100t "13631093"
set code [drscan xc7.tap 32 0]  
puts $code

if { $code == $xc7a35t} {
    puts "The board has an xc7a35t"
}

if { $code == $xc7a100t} {
    puts "The board has an xc7a100t"
}

puts "Programming..."
"""
+
f"""
pld load 0 {bitfile}.bit
exit"""
)

In [ ]:
#@title Create Zip
%%bash
cd /content
export FOLDER=State_Machine
cp $FOLDER/build/basys3/*.bit ./
zip $FOLDER.zip *.bit $FOLDER.cfg